In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
from einops import rearrange, repeat, reduce
from pytorch_wavelets import DWTForward, DWTInverse

In [3]:
level = 2
wavelet = 'haar'
dwt1 = DWTForward(J=level, wave=wavelet, mode='zero').cuda()
dwt2 = DWTForward(J=level-1, wave=wavelet, mode='zero').cuda()

In [8]:
x = torch.rand(1, 3, 256, 256).cuda()
x_y = x[:,0:1]
x_cr = x[:,1:2]
x_cb = x[:,2:]
shape = x_cr.shape[2]
x_y.shape, x_cr.shape, x_cb.shape
x_cr = F.interpolate(x_cr, size=shape//2, mode="bicubic", align_corners=True)
x_cb = F.interpolate(x_cb, size=shape//2, mode="bicubic", align_corners=True)
print(x_cr.shape, x_cb.shape)

y = dwt1(x_y)
cr = dwt2(x_cr)
cb = dwt2(x_cb)

(torch.Size([1, 1, 256, 256]),
 torch.Size([1, 1, 256, 256]),
 torch.Size([1, 1, 256, 256]))

In [14]:
# merge tensor
# ll = rearrange(y[0], "n (b c) h w -> n b c h w", b=1, c=1)

ll = y[0]
h0 = rearrange(y[1][0], "n b c (d h) (e w) -> n (b c d e) h w", d =2, e =2)
h1 = rearrange(y[1][1], "n b c h w -> n (b c) h w")
yy = torch.cat([ll, h0, h1], dim = 1)
# merge tensor
# ll = rearrange(y[0], "n (b c) h w -> n b c h w", b=1, c=1)
ll = cr[0]
h0 = rearrange(cr[1][0], "n b c h w -> n (b c) h w")
ycr = torch.cat([ll, h0], dim = 1)
# merge tensor
# ll = rearrange(y[0], "n (b c) h w -> n b c h w", b=1, c=1)
ll = cb[0]
h0 = rearrange(cb[1][0], "n b c h w -> n (b c) h w")
ycb = torch.cat([ll, h0], dim = 1)
o = torch.cat([yy, ycr, ycb], dim = 1)

In [11]:
print(len(y))
print(y[0].shape, len(y[1]))
print(y[1][0].shape, y[1][1].shape)
print(len(cr))
print(cr[0].shape, len(cr[1]))
print(cr[1][0].shape)
print(len(cb))
print(cb[0].shape, len(cb[1]))
print(cb[1][0].shape)

2
torch.Size([1, 1, 64, 64]) 2
torch.Size([1, 1, 3, 128, 128]) torch.Size([1, 1, 3, 64, 64])
2
torch.Size([1, 1, 64, 64]) 1
torch.Size([1, 1, 3, 64, 64])
2
torch.Size([1, 1, 64, 64]) 1
torch.Size([1, 1, 3, 64, 64])


In [15]:
o.shape

torch.Size([1, 24, 64, 64])